In [12]:
#spark.stop()

In [1]:
import os, sys
import datetime as dt
DEVEL_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src", "devel")
if DEVEL_SRC not in sys.path:
    sys.path.append(DEVEL_SRC)

USECASES_SRC = os.path.join(DEVEL_SRC, "use-cases") # TODO when - is removed, remove also this line and adapt imports
if USECASES_SRC not in sys.path: 
    sys.path.append(USECASES_SRC)
    
AMDOCS_SRC = os.path.join(DEVEL_SRC, "amdocs_informational_dataset") # TODO when - is removed, remove also this line and adapt imports
if AMDOCS_SRC not in sys.path: 
    sys.path.append(AMDOCS_SRC)
    
import pykhaos.utils.custom_logger as clogger
logging_file = os.path.join(os.environ.get('BDA_USER_HOME', ''), "logging",
                                    "out_" + dt.datetime.now().strftime("%Y%m%d_%H%M%S") + ".log")
logger = clogger.configure_logger(log_filename=logging_file, std_channel=sys.stderr, logger_name="")
logger.info("Logging to file {}".format(logging_file))    
    
    
from project.project_generic import Project


import pykhaos.utils.notebooks as nb

project_obj = Project("CCC_model_nb", "CCC_model_nb")   

RUNNING_FROM_NOTEBOOK = nb.isnotebook()
import matplotlib.pyplot as plt
if RUNNING_FROM_NOTEBOOK:
    %load_ext autoreload
    %autoreload 2
    %matplotlib inline  
    
#logger = my_project.logger

if not RUNNING_FROM_NOTEBOOK:
    args = my_project.arg_parser()

20181207-141032 [INFO ] Logging to file /var/SP/data/home/csanc109/logging/out_20181207_141032.log


_initialize spark
Ended spark session: 908.770467043 secs | default parallelism=2


In [91]:
import os
os.environ["SPARK_COMMON_OPTS"]

" --master yarn --conf spark.serializer=org.apache.spark.serializer.KryoSerializer --conf spark.kryoserializer.buffer.max=1g --py-files /var/SP/data/home/csanc109/artifacts/bda-core-ra-complete-assembly-2.0.0.jar,/var/SP/data/home/csanc109/artifacts/common.zip,/var/SP/data/home/csanc109/artifacts/graphframes.zip,/var/SP/data/home/csanc109/artifacts/scripts.zip,/var/SP/data/home/csanc109/artifacts/xgboost4j-spark-2.1.1-0.7-jar-with-dependencies.jar --jars /var/SP/data/home/csanc109/artifacts/bda-core-ra-complete-assembly-2.0.0.jar,/var/SP/data/home/csanc109/artifacts/xgboost4j-spark-2.1.1-0.7-jar-with-dependencies.jar --files /var/SP/data/home/csanc109/scripts/properties/red_agent/nodes-cz.yaml,/var/SP/data/home/csanc109/scripts/properties/red_agent/nodes-de.yaml,/var/SP/data/home/csanc109/scripts/properties/red_agent/nodes-es.yaml,/var/SP/data/home/csanc109/scripts/properties/red_agent/nodes-gr.yaml,/var/SP/data/home/csanc109/scripts/properties/red_agent/nodes-ie.yaml,/var/SP/data/home

In [2]:
spark=project_obj.spark

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
from amdocs_informational_dataset.engine.call_centre_calls import CallCentreCalls
from pyspark.sql.functions import collect_set, concat, size, coalesce, col, lpad, struct, count as sql_count, lit, min as sql_min, max as sql_max, collect_list, udf, when
from pyspark.sql.types import StringType, ArrayType, MapType, StructType, StructField, IntegerType
from pyspark.sql.functions import array, regexp_extract

In [4]:

from churn.datapreparation.data_loader import get_port_requests_table
from churn.datapreparation.config import Config

DEBUG=True

port_start_date = "20181001"
port_end_date = "20181031"
ref_date = port_end_date

PORTOUT_TABLE_COLS = ["msisdn_a", "portout_date"] # ["portout_date",  "msisdn_a", "label", "days_from_portout"]

ccc_start_model = "20180801"
ccc_end_model = "20180930"

n_ccc = -60 # FIXME -abs(config_obj.['data_preparation'][60])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from churn.config_manager.ccc_model_config_mgr import  CCCmodelConfig
config_obj = CCCmodelConfig('/var/SP/data/home/csanc109/src/devel/use-cases/churn/input/dp_ccc_model.yaml')


Reading config from file /var/SP/data/home/csanc109/src/devel/use-cases/churn/input/dp_ccc_model.yaml
Reading internal config from file /var/SP/data/home/csanc109/src/devel/use-cases/churn/config_manager/config/internal_config_ccc_model.yaml
----- CHECKING INPUT PARAMETERS CCCmodel------
{'ccc_days': -60,
 'closing_day': 20180531,
 'end_port': 20180531,
 'internal_config_file': '/var/SP/data/home/csanc109/src/devel/use-cases/churn/config_manager/config/internal_config_ccc_model.yaml',
 'labeled': True,
 'level': 'nif',
 'model_target': 'comercial',
 'save_car': True,
 'sources': {'ids': {'address': False,
                     'billing': True,
                     'call_centre_calls': True,
                     'campaigns': False,
                     'customer': True,
                     'customer_aggregations': True,
                     'customer_penalties': False,
                     'device_catalogue': False,
                     'geneva': False,
                     'netscout': 

In [32]:
from pyspark.sql.functions import col

from amdocs_informational_dataset.engine.call_centre_calls import CallCentreCalls

from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import col, concat, concat_ws, date_format, dayofmonth, format_string, from_unixtime, length, \
	lit, lower, lpad, month, regexp_replace, translate, udf, unix_timestamp, year, when, upper, collect_set, collect_list, \
    count as sql_count, min as sql_min, max as sql_max, struct, size, coalesce, sum as sql_sum
from pyspark.sql.types import DateType, IntegerType, StringType, StructField, StructType
from pyspark.sql.utils import AnalysisException
from engine.general_functions import format_date, compute_diff_days, sum_horizontal
from collections import Counter
from pyspark.sql.types import StringType


CAT_PRECIO = "PRECIO"
CAT_TERMINAL = "TERMINAL"
CAT_CONTENIDOS = "CONTENIDOS"
CAT_COMERCIAL = "COMERCIAL"
CAT_NO_COMERCIAL = "NO_COMERCIAL"
CAT_SERV_ATENCION = "SERVICIO/ATENCION"
CAT_TECNICO = "TECNICO"
CAT_BILLING = "BILLING"
CAT_FRAUD = "FRAUD"
NO_PROB = "NO_PROB"
NA = "NA"

# ADD CATEGORIES TO CALLS
REASON1_PRECIO = ["Cliente quiere pagar menos", "Clte quiere pagar menos"]



def __aggregate_ccc_calls_by_nif(df_all, process_date):

    print("Calling to __aggregate_ccc_calls_by_nif.... {}".format(process_date))

    df_agg = (df_all
              .withColumn("fx_interaction",
                          concat(col('year'), lpad(col('month'), 2, '0'), lpad(col('day'), 2, '0')))
              .withColumn("tuples", struct(["fx_interaction", "Bucket"]))
              .groupby("NIF")
              .agg(sql_count(lit(1)).alias("num_interactions"),
                   sql_min(col('fx_interaction')).alias("first_interaction"),
                   sql_max(col('fx_interaction')).alias("latest_interaction"),
                   sql_max(col("tuples")).alias("tuples_max"),  # latest interaction: [max(fx_interaction), bucket]
                   sql_min(col("tuples")).alias("tuples_min"),  # first interaction: [min(fx_interaction), bucket]
                   collect_list('Bucket').alias("bucket_list"),
                   collect_set('Bucket').alias("bucket_set"),
                   sql_sum("Bucket_NA").alias("num_NA_buckets"),
                   sql_sum("IVR").alias("num_ivr_interactions"),
                   collect_list('CATEGORY_1').alias("cat1_list"),
                   collect_list('CATEGORY_2').alias("cat2_list"),
            ))

    df_agg = df_agg.withColumn("ref_date", format_date(lit(process_date)))
    df_agg = (df_agg.withColumn("bucket_1st_interaction", col("tuples_min")["Bucket"])
              .withColumn("bucket_latest_interaction", col("tuples_max")["Bucket"])
              .withColumn("nb_diff_buckets", size("bucket_set"))
              .drop(*['tuples_max', 'tuples_min'])
              )

    for cc in ["first_interaction", "latest_interaction"]:
        df_agg = (df_agg.withColumn("fx_{}".format(cc),
                                    format_date(cc, filter_dates_1900=True))  # days before 1900 converted to None
                  .withColumn("days_since_{}".format(cc), compute_diff_days("fx_{}".format(cc), "ref_date")))

    df_agg = df_agg.drop("ref_date")

    def get_mode_problems(lst):
        # filter NA y NO_PROB
        if not lst: return None
        lst=[ll for ll in lst if ll and ll!=NO_PROB]
        if not lst: return None
        dd = Counter(lst).most_common(2)
        return dd[0][0]
    get_mode_udf = udf(lambda lst: get_mode_problems(lst), StringType())

    df_agg = (df_agg.withColumn("CAT1_MODE", when(coalesce(size(col("cat1_list")), lit(0)) == 0, "None").otherwise(get_mode_udf(col("cat1_list"))))
              .withColumn("CAT2_MODE", when(coalesce(size(col("cat1_list")), lit(0)) == 0, "None").otherwise(get_mode_udf(col("cat2_list")))))

    return df_agg


def __get_agg_ccc_nif(spark, ccc_start_model, ccc_end_model, msisdn_list=None):
    '''
    msisdn_list a list of msisdn's to filter the calls. Usually, this list corresponds to portouts
    '''
    ccc = CallCentreCalls(spark)
    ccc.prepareFeatures(ccc_end_model, ccc_start_model)
    df_all = ccc.all_interactions
    if msisdn_list:
        df_all = df_all.where(col("msisdn").isin(msisdn_list))
        #df_ccc = df_ccc.where(col("msisdn").isin(msisdn_list))
    df_all = __set_categories(df_all)
    df_agg = __aggregate_ccc_calls_by_nif(df_all, ccc_end_model)
    return df_agg


def __set_categories(df_all):

    df_all = df_all.withColumn("is_fraud", when(col("INT_RAZON").rlike("(?i)enga.o comercial|venta enga.osa") ,1).otherwise(0))
    df_all = df_all.withColumn("Queja_Trato", when(col("INT_SUBTIPO").isin(["Queja trato", "Quejas sevicios de atencion"])
                                    ,1).otherwise(0))

    df_all = df_all.withColumn("CATEGORY_2",
                               # - - - - - - - - - PRECIO
                               when(col('INT_TIPO').isin(REASON1_PRECIO) , CAT_PRECIO)
                               # - - - - - - - - - SERVICIO/ATENCION
                               .when(lower(col("INT_SUBTIPO")).isin
                                   (["Queja trato".lower(), "Quejas sevicios de atencion".lower()]) ,CAT_SERV_ATENCION)
                               # - - - - - - - - - ENGAÑO
                               .when(col("INT_RAZON").rlike("(?i)enga.o comercial|venta enga.osa"), CAT_FRAUD)
                               # - - - - - - - - - TECNICO
                               .when(col('INT_TIPO').rlike('(?i)^Averia') ,CAT_TECNICO)
                               .when((col('INT_TIPO').rlike('(?i)^Transferencia') )&
                                   (col("INT_SUBTIPO").rlike('(?i)^Aver.as')), CAT_TECNICO)
                               .when(col('INT_TIPO').rlike(
                                   '(?i)^Inc Provis.*Neba|^Inc Provision Fibra|^Inc Provision DSL|^Incidencia%ecnica|^Incidencia%SGI|^Inc|^Incidencia'),
                                     CAT_TECNICO)
                               .when(col('INT_TIPO').rlike('(?i)^Consulta tec'), CAT_TECNICO)
                               .when(col("Raw_Resultado").rlike(
                                   "Raw_Resultado_Escalo|Raw_Resultado_Envio_tecnico|Raw_Resultado_Transferencia|Raw_Resultado_Reclamacion"),
                                     CAT_TECNICO)
                               # - - - - - - - - - FACTURA
                               .when(col('INT_TIPO').rlike('(?i)factura'), CAT_BILLING)
                               # - - - - - - - - - TERMINAL
                               .when(col("INT_TIPO") == "TERMINAL", CAT_TERMINAL)
                               # - - - - - - - - - NO_PROB :)
                               .when((col("INT_TIPO") == "INFORMACION") | ((col("INT_TIPO") == "TRANSFERENCIA") & (
                                           col("Bucket") == "Other customer information management")),
                                     NO_PROB).otherwise(NA)
                               )  # end

    df_all = df_all.na.fill(NA, subset=["CATEGORY_2"])

    df_all = (df_all.withColumn("CATEGORY_1",
                                when(col('CATEGORY_2').isin([CAT_PRECIO, CAT_TERMINAL, CAT_CONTENIDOS]), CAT_COMERCIAL)
                                .when(col('CATEGORY_2').isin([CAT_SERV_ATENCION, CAT_TECNICO, CAT_BILLING, CAT_FRAUD]),
                                      CAT_NO_COMERCIAL)
                                .otherwise(col('CATEGORY_2'))))
    # FALTA
    # col('Raw_Productos') hay que desglosarlo
    return df_all


def check_partitions (df):
    l = df.rdd.glom().map(len).collect()  # get length of each partition
    print 'Smallest partition {}'.format(min(l))
    print 'Largest partitions {}'.format(max(l))
    print 'Avg. partition size {}'.format(sum(l)/len(l))
    print 'Num partitions {}'.format(len(l))


In [29]:
from pykhaos.utils.pyspark_utils import union_all
from pykhaos.utils.date_functions import move_date_n_days

from churn.datapreparation.general.data_loader import get_port_requests_table
n_ccc= config_obj.get_ccc_range_duration()
portout_select_cols = ["msisdn_a", "portout_date"]  # ["portout_date",  "msisdn_a", "label", "days_from_portout"]


df_port = get_port_requests_table(spark, config_obj, ref_date=None,
                                  select_cols=portout_select_cols)

portout_dates_list = df_port.select("portout_date").distinct().rdd.map(lambda x: x["portout_date"]).collect()

from pykhaos.utils.pyspark_utils import union_all
from pykhaos.utils.date_functions import move_date_n_days
from pyspark.sql.functions import col

dfs = []
import time

from churn.datapreparation.general.ccc_data_loader import get_calls_for_portouts, get_calls_for_portouts_filename
from pyspark.sql.functions import concat_ws
calls_for_portouts_filename = get_calls_for_portouts_filename(config_obj)
from pykhaos.utils.hdfs_functions import check_hdfs_exists
if not check_hdfs_exists(calls_for_portouts_filename):
    if logger: logger.info("'{}' does not exist!".format(calls_for_portouts_filename))
    df_agg_all_ccc = get_calls_for_portouts(spark, config_obj, ref_date=None, portout_select_cols=["msisdn_a", "portout_date"], logger=logger)
    #if logger: logger.info("[Info ccc_model_data_loader] Number of NIFs df_agg_all_ccc: df_agg_all_ccc={} ".format(df_agg_all_ccc.count()))
    
    start_time = time.time()
    #check_partitions(df_agg_all_ccc)

    df_agg_all_ccc = df_agg_all_ccc.withColumn('cat1_list', concat_ws(",", col('cat1_list')))
    df_agg_all_ccc = df_agg_all_ccc.withColumn('cat2_list', concat_ws(",", col('cat2_list')))
    df_agg_all_ccc = df_agg_all_ccc.drop('bucket_list', 'bucket_set')
    
    print("Elapsed {} minutes".format((time.time()-start_time)/60.0))
    
#     print(df_agg_all_ccc.dtypes)
#     if logger: logger.info("Prepared to write to {}".format(calls_for_portouts_filename))
    #df_agg_all_ccc.write.mode('overwrite').format('csv').option('sep', '|').option('header', 'true').save(calls_for_portouts_filename)
# else:
#     if logger: logger.info("'{}' exists! Loading!".format(calls_for_portouts_filename))
#     df_agg_all_ccc = spark.read.option("delimiter", "|").option("header", True).csv(calls_for_portouts_filename)

# if logger: logger.info("[Info ccc_model_data_loader] Number of NIFs df_agg_all_ccc: df_agg_all_ccc={} ".format(df_agg_all_ccc.count()))

# if config_obj.get_model_target() == "comercial":
#     df_agg_all_ccc = df_agg_all_ccc.withColumn("label", when(col("CAT1_MODE")=="COMERCIAL", 1).when(col("CAT1_MODE")=="NO_COMERCIAL", 0).otherwise(-1))
# else:
#     if logger: logger.error("Model target {} is not implemented".format(config_obj.get_model_target()))
#     sys.exit()

#check_partitions(df_agg_all_ccc)

#if logger: logger.info("Number of rows {} (each row is a NIF)".format(df_agg_all_ccc.count()))


Get port requests table: start=20180501 end=20180531 ref_date=20180531


20181207-191652 [INFO ] '/user/csanc109/projects/churn/data/ccc_model/df_calls_for_portouts_nif_20180501_20180531_60' does not exist!


Get port requests table: start=20180501 end=20180531 ref_date=20180531


20181207-191658 [INFO ] df_port=177171


Elapsed time _set_categories = 0.00680894851685 minutes
Calling to __aggregate_ccc_calls_by_nif.... 20180530
Elapsed time __aggregate_ccc_calls_by_nif = 0.0564715027809 minutes
Elapsed 5.16573588053e-08 minutes


In [34]:
#check_partitions(df_agg_all_ccc)
df_agg_all_ccc = df_agg_all_ccc.repartition(200)



In [36]:
df_agg_all_ccc = df_agg_all_ccc.withColumn('cat1_list', concat_ws(",", col('cat1_list')))
df_agg_all_ccc = df_agg_all_ccc.withColumn('cat2_list', concat_ws(",", col('cat2_list')))
df_agg_all_ccc = df_agg_all_ccc.drop('bucket_list', 'bucket_set')

In [38]:
print(calls_for_portouts_filename)
#calls_for_portouts_filename = '/user/csanc109/projects/churn/data/ccc_model/df_calls_for_portouts_nif_20180501_20180630_60'
from pykhaos.utils.hdfs_functions import check_hdfs_exists
check_hdfs_exists(calls_for_portouts_filename)
df_agg_all_ccc.write.mode('overwrite').format('csv').option('sep', '|').option('header', 'true').save(calls_for_portouts_filename)


/user/csanc109/projects/churn/data/ccc_model/df_calls_for_portouts_nif_20180501_20180531_60


KeyboardInterrupt: 

# PROBANDO CATEGORIAS

In [ ]:
df_all.where(col("CATEGORY_1")=="NA").select("INT_Tipo", "INT_Subtipo","INT_Resultado").groupBy("INT_Tipo", "INT_Tipo","INT_Resultado").agg(sql_count("*").alias("count")).sort(desc("count")).show(30, False)

In [319]:
df_agg_all_ccc.select("CAT1_MODE").groupBy("CAT1_MODE").agg(sql_count("*").alias("count")).sort(desc("count")).show(30, False)

KeyboardInterrupt: 

In [13]:
df_agg_all_ccc.select("CAT2_MODE").groupBy("CAT2_MODE").agg(sql_count("*").alias("count")).sort(desc("count")).show(30, False)

NameError: name 'desc' is not defined

REVISANDO RESULTADOS ENCUESTAS

In [8]:
from churn.utils.general_functions import get_nif_msisdn
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, collect_set, concat, size, coalesce, col, lpad, struct, count as sql_count, regexp_replace, lit, min as sql_min, max as sql_max, collect_list, udf, when, desc, row_number
from churn.datapreparation.engine.bajas_data_loader import load_reasons_encuestas

In [31]:
calls_for_portouts_filename = '/user/csanc109/projects/churn/data/ccc_model/df_calls_for_portouts_filename_nif_encuestas_20180501_20180630_60'
df_agg_ccc_encuestas = spark.read.option("delimiter", "|").option("header", True).csv(calls_for_portouts_filename)

In [35]:
df_agg_ccc_encuestas.columns

['NIF',
 'num_interactions',
 'first_interaction',
 'latest_interaction',
 'num_NA_buckets',
 'num_ivr_interactions',
 'cat1_list',
 'cat2_list',
 'bucket_1st_interaction',
 'bucket_latest_interaction',
 'nb_diff_buckets',
 'fx_first_interaction',
 'days_since_first_interaction',
 'fx_latest_interaction',
 'days_since_latest_interaction',
 'CAT1_MODE',
 'CAT2_MODE',
 'REASON_ENCUESTA']

In [45]:
ccc = CallCentreCalls(spark)
ccc.prepareFeatures("20180601", "20180301")
df_all = ccc.all_interactions
df_all2 = df_all.where(col("NIF").isin(["67302557O"])) 

In [46]:
df_all2.show(100,False)

+---------+---------+-----------------+----+-----+---+-------------+---------------+----------------------+-------------+----------+-----------------+------+-------+--------------------------------+---------------+---------+-----------+----------+--------+-----------------+-----------+----------------+----------------------------------------------------------+--------------+------------+---------------+----------------+----------+-------------+-------------------------+-------------+-------------------------+-----------------+--------+---+---------+
|nif      |msisdn   |partitioned_month|year|month|day|INT_Tipo     |INT_Subtipo    |INT_Razon             |INT_Resultado|DIRECTION |x_workgroup      |source|TYPE_TD|Sub_Bucket                      |Raw_Pagar_menos|Raw_Cobro|Raw_Precios|Raw_Averia|Raw_Alta|Raw_Desactivacion|Raw_Factura|Raw_Ofrecimiento|Bucket_Sub_Bucket                                         |Raw_Incidencia|Raw_Consulta|Raw_Informacion|Raw_Portabilidad|Raw_Cierre|Raw_Produc

In [55]:
df_agg_ccc_encuestas.where( (col("REASON_ENCUESTA")=="FRAUD") & (col("num_interactions").isNotNull())).select("NIF", "CAT2_MODE", "cat2_list", "cat1_list", "num_interactions").show(100, False)

Py4JJavaError: An error occurred while calling o2921.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 262.0 failed 4 times, most recent failure: Lost task 0.3 in stage 262.0 (TID 12018, vgddp367hr.dc.sedc.internal.vodafone.com, executor 31): java.io.FileNotFoundException: File does not exist: /user/csanc109/projects/churn/data/ccc_model/df_calls_for_portouts_filename_nif_encuestas_20180501_20180630_60/part-00067-4d72064d-cd6d-4b3b-9e0a-398e595deb29.csv
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:66)
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:56)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getBlockLocationsInt(FSNamesystem.java:2007)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getBlockLocations(FSNamesystem.java:1977)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getBlockLocations(FSNamesystem.java:1890)
	at org.apache.hadoop.hdfs.server.name

In [33]:
df_agg_ccc_encuestas.select("CAT2_MODE", "REASON_ENCUESTA").groupBy("REASON_ENCUESTA", "CAT2_MODE").agg(sql_count("*").alias("count")).sort(desc("count")).show(150, False)

+-----------------+-----------------+-----+
|REASON_ENCUESTA  |CAT2_MODE        |count|
+-----------------+-----------------+-----+
|null             |null             |95221|
|null             |BILLING          |15921|
|null             |TECNICO          |10580|
|null             |TERMINAL         |2408 |
|PRECIO           |null             |2264 |
|SERVICIO/ATENCION|null             |1175 |
|FRAUD            |null             |928  |
|SITUACION        |null             |666  |
|CONTENIDOS       |null             |662  |
|null             |FRAUD            |293  |
|BILLING          |null             |268  |
|null             |SERVICIO/ATENCION|78   |
|OTROS            |null             |52   |
|NO_FIBRA         |null             |10   |
|SERVICIO/ATENCION|TECNICO          |3    |
|PRECIO           |BILLING          |3    |
|SERVICIO/ATENCION|BILLING          |2    |
|PRECIO           |FRAUD            |1    |
|PRECIO           |TECNICO          |1    |
|FRAUD            |BILLING      

# WRITER

In [ ]:
llist = [('bob', '2015-01-13', 4), ('alice', '2015-04-23',10)]
left = spark.createDataFrame(llist, ['name','date','duration'])
right = spark.createDataFrame([('fffff', 100),('bob', 23)],['name','upload'])

In [52]:
[col_ for col_ in df.columns if col_.endswith("nif")]

['fbb_services_nif',
 'fbb_fx_first_nif',
 'mobile_services_nif',
 'mobile_fx_first_nif',
 'tv_services_nif',
 'tv_fx_first_nif',
 'prepaid_services_nif',
 'prepaid_fx_first_nif',
 'bam_mobile_services_nif',
 'bam_mobile_fx_first_nif',
 'fixed_services_nif',
 'fixed_fx_first_nif',
 'bam_services_nif',
 'bam_fx_first_nif',
 'flag_prepaid_nif',
 'seg_pospaid_nif',
 'num_football_nif',
 'total_football_price_nif',
 'total_num_services_nif',
 'fx_bam_fx_first_nif',
 'days_since_bam_fx_first_nif',
 'fx_bam_mobile_fx_first_nif',
 'days_since_bam_mobile_fx_first_nif',
 'fx_fbb_fx_first_nif',
 'days_since_fbb_fx_first_nif',
 'fx_fixed_fx_first_nif',
 'days_since_fixed_fx_first_nif',
 'fx_mobile_fx_first_nif',
 'days_since_mobile_fx_first_nif',
 'fx_prepaid_fx_first_nif',
 'days_since_prepaid_fx_first_nif',
 'fx_tv_fx_first_nif',
 'days_since_tv_fx_first_nif']

In [53]:
tablename = "/data/udf/vf_es/churn/ccc_model/comercial/df_20180501_20180531_n60_comercial"
df = spark.read.option("delimiter", "|").option("header", True).csv(tablename)
#df = spark.read.load(tablename)
df = df.where(col("label")!=-1)
#df = df.repartition(1)
#df.write.mode('overwrite').format('csv').option('sep', '|').option('header', 'true').save("/data/udf/vf_es/churn/ccc_model/comercial/df_20180501_20180630_n60_comercial_csv")
#len(df.columns)

In [83]:
from pykhaos.utils.hdfs_functions import move_hdfs_dir_to_local

move_hdfs_dir_to_local("hdfs://vgddp349hr.dc.sedc.internal.vodafone.com/data/udf/vf_es/churn/ccc_model/comercial/df_20180501_20180531_n60_comercial/", "/var/SP/data/home/csanc109/tmp/kk/")

Running system command: hdfs dfs -copyToLocal hdfs://vgddp349hr.dc.sedc.internal.vodafone.com/data/udf/vf_es/churn/ccc_model/comercial/df_20180501_20180531_n60_comercial/ /var/SP/data/home/csanc109/tmp/kk/


1

In [88]:
df = h2o.import_file("hdfs://vgddp349hr.dc.sedc.internal.vodafone.com/data/udf/vf_es/churn/ccc_model/comercial/df_20180501_20180531_n60_comercial", pattern = ".*\.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [87]:
from pykhaos.modeling.h2o.h2o_functions import import_file_to_h2o_frame

df_h2o_imported = import_file_to_h2o_frame("/data/udf/vf_es/churn/ccc_model/comercial/df_20180501_20180531_n60_comercial", hdfs=True, parquet=False)

('HOLAAAAA', <logging.RootLogger object at 0x7f2b7898ba90>)


20181208-121935 [INFO ] ***** locked=False healthy=True
20181208-121935 [INFO ] Trying to import '/data/udf/vf_es/churn/ccc_model/comercial/df_20180501_20180531_n60_comercial' hdfs=True parquet=False
20181208-121937 [INFO ] Trying to import hdfs://vgddp349hr.dc.sedc.internal.vodafone.com/data/udf/vf_es/churn/ccc_model/comercial/df_20180501_20180531_n60_comercial


Parse progress: |█████████████████████████████████████████████████████████| 100%


20181208-121937 [INFO ] file hdfs://vgddp349hr.dc.sedc.internal.vodafone.com/data/udf/vf_es/churn/ccc_model/comercial/df_20180501_20180531_n60_comercial found at node 0


In [ ]:
20181208-122104 [INFO ] Trying to import '/data/udf/vf_es/churn/ccc_model/comercial/df_20180501_20180531_n60_comercial' hdfs=True parquet=False
20181208-122105 [INFO ] Trying to import hdfs://vgddp349hr.dc.sedc.internal.vodafone.com/data/udf/vf_es/churn/ccc_model/comercial/df_20180501_20180531_n60_comercial
20181208-122105 [ERROR] file hdfs://vgddp349hr.dc.sedc.internal.vodafone.com/data/udf/vf_es/churn/ccc_model/comercial/df_20180501_20180531_n60_comercial could not be found at node 0


In [70]:
import h2o
#h2o.cluster().shutdown()

h2o.init(nthreads=-1, port="54323", max_mem_size="6g")

# from pykhaos.utils.hdfs_functions import move_hdfs_dir_to_local, move_hdfs_file_to_local

# #local_filename = move_hdfs_file_to_local('/data/udf/vf_es/churn/ccc_model/comercial/df_20180501_20180630_n60_comercial_csv/', "/var/SP/data/home/csanc109/tmp/")
# print(local_filename)
# df_h2o_imported = h2o.import_file(local_filename, pattern = ".*\.csv") #, sep = "|",header=1)

Checking whether there is an H2O instance running at http://localhost:54323..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.7.0_79"; Java(TM) SE Runtime Environment (build 1.7.0_79-b15); Java HotSpot(TM) 64-Bit Server VM (build 24.79-b02, mixed mode)
  Starting server from /opt/cloudera/parcels/Anaconda-2.5.0/lib/python2.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpw07zOW
  JVM stdout: /tmp/tmpw07zOW/h2o_csanc109_started_from_python.out
  JVM stderr: /tmp/tmpw07zOW/h2o_csanc109_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Rome
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.10
H2O cluster version age:,6 months and 16 days !!!
H2O cluster name:,H2O_from_python_csanc109_6wbgof
H2O cluster total nodes:,1
H2O cluster free memory:,5.333 Gb
H2O cluster total cores:,56
H2O cluster allowed cores:,56
H2O cluster status:,"accepting new members, healthy"


In [39]:
closing_day = "20180531"
table_name = "all_amdocs_inf_dataset"
print("amdocs_table_reader", table_name, closing_day)
table_name = '/data/udf/vf_es/amdocs_inf_dataset/{}/year={}/month={}/day={}'.format(table_name, int(closing_day[:4]),
                                                                                                    int(closing_day[4:6]),
                                                                                                    int(closing_day[6:]))

df_src = spark.read.load(table_name)


('amdocs_table_reader', 'all_amdocs_inf_dataset', '20180531')


AnalysisException: u'Path does not exist: hdfs://nameservice1/data/udf/vf_es/amdocs_inf_dataset/all_amdocs_inf_dataset/year=2018/month=5/day=31;'